In [1]:
import sys
sys.path.append("../../src")
sys.path.append("../../../../..")
import os
import datetime
import pandas as pd
import numpy as np
from example_reactiondiffusion import get_rd_data
from sindy_utils import library_size
from training import train_network
import tensorflow as tf
import time

2025-05-29 09:28:18.082229: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748525298.106616 2638364 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748525298.114157 2638364 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-05-29 09:28:18.140794: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Instructions for updating:
non-resource variables are not supported in the long term


# Generate data

In [2]:
training_data, validation_data, test_data = get_rd_data()

In [3]:

print("Training data key:", training_data.keys())
print("Training data t shape:", training_data['t'].shape)
print("Training data x shape:", training_data['x'].shape)
print("Training data dx shape:", training_data['dx'].shape)
print("Training data y1 shape:", training_data['y1'].shape)
print("Training data y2 shape:", training_data['y2'].shape)
print("Validation data key:", validation_data.keys())
print("Test data key:", test_data.keys())

Training data key: dict_keys(['t', 'y1', 'y2', 'x', 'dx'])
Training data t shape: (7999, 1)
Training data x shape: (7999, 10000)
Training data dx shape: (7999, 10000)
Training data y1 shape: (100, 1)
Training data y2 shape: (100, 1)
Validation data key: dict_keys(['t', 'y1', 'y2', 'x', 'dx'])
Test data key: dict_keys(['t', 'y1', 'y2', 'x', 'dx'])


# Set up model and training parameters

In [4]:
params = {}

params['input_dim'] = training_data['y1'].size*training_data['y2'].size
params['latent_dim'] = 2
params['model_order'] = 1
params['poly_order'] = 3
params['include_sine'] = True
params['library_dim'] = library_size(params['latent_dim'], params['poly_order'], params['include_sine'], True)

# sequential thresholding parameters
params['sequential_thresholding'] = True
params['coefficient_threshold'] = 0.1
params['threshold_frequency'] = 500
params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))
params['coefficient_initialization'] = 'constant'

# loss function weighting
params['loss_weight_decoder'] = 1.0
params['loss_weight_sindy_z'] = 0.01
params['loss_weight_sindy_x'] = 0.5
params['loss_weight_sindy_regularization'] = 0.1

params['activation'] = 'sigmoid'
params['widths'] = [256]

# training parameters
params['epoch_size'] = training_data['t'].size
params['batch_size'] = 1000
params['learning_rate'] = 1e-3

params['data_path'] = os.getcwd() + '/'
params['print_progress'] = True
params['print_frequency'] = 100

# training time cutoffs
params['max_epochs'] = 3001
params['refinement_epochs'] = 1001

# Run training experiments

In [5]:
start_time = time.time()
num_experiments = 1
df = pd.DataFrame()
for i in range(num_experiments):
    print('EXPERIMENT %d' % i)

    params['coefficient_mask'] = np.ones((params['library_dim'], params['latent_dim']))

    params['save_name'] = 'model_noisy'#'rd_' + datetime.datetime.now().strftime("%Y_%m_%d_%H_%M_%S_%f")

    tf.compat.v1.reset_default_graph()

    results_dict = train_network(training_data, validation_data, params)
    #df = pd.concat([df, {**results_dict, **params}])
    new_row = pd.DataFrame([{**results_dict, **params}])
    df = pd.concat([df, new_row], ignore_index=True)

    print('Experiment %d completed in %.2f seconds' % (i, time.time() - start_time))
df.to_pickle('experiment_results_' + datetime.datetime.now().strftime("%Y%m%d%H%M") + '.pkl')

EXPERIMENT 0
TRAINING


I0000 00:00:1748525338.898948 2638364 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18934 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:82:00.0, compute capability: 8.9
I0000 00:00:1748525338.927709 2638364 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled


Epoch 0
   training loss 5.075778961181641, (0.16202483, 474.169, 0.14433075, 0.99898535)
   validation loss 5.137853145599365, (0.16235298, 480.33524, 0.14449866, 0.99898535)
decoder loss ratio: 1.024982, decoder SINDy loss  ratio: 1.105488
Epoch 100
   training loss 0.2377086579799652, (0.084470026, 1.8683292, 0.073317625, 0.9789653)
   validation loss 0.23892737925052643, (0.085259676, 1.9381658, 0.07277902, 0.9789653)
decoder loss ratio: 0.538269, decoder SINDy loss  ratio: 0.556797
Epoch 200
   training loss 0.2043123096227646, (0.068523966, 0.8636147, 0.06489827, 0.9470306)
   validation loss 0.20611104369163513, (0.06945649, 0.87652236, 0.06637253, 0.9470306)
decoder loss ratio: 0.438499, decoder SINDy loss  ratio: 0.507784
Epoch 300
   training loss 0.1773369312286377, (0.054198507, 0.5393707, 0.054120626, 0.9068441)
   validation loss 0.18002718687057495, (0.055611443, 0.53997606, 0.056663156, 0.9068441)
decoder loss ratio: 0.351091, decoder SINDy loss  ratio: 0.433502
Epoch 4